In [2]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
import serial
import time
from picoscope import ps5000a
import picosdk
from picosdk.discover import find_all_units
import serial.tools.list_ports as port_list
import chipwhisperer as cw

# Check if the Picoscope is connected
scopes = find_all_units()
for scope in scopes:
    print("Working with:")
    print(scope.info)
    scope.close()
ports = list(port_list.comports())
for p in ports:
    print (p)


ModuleNotFoundError: No module named 'chipwhisperer'

**PicoScope setup**

In [2]:
ps = ps5000a.PS5000a()
print("Found the following picoscope:")
print(ps.getAllUnitInfo())

# Since target runnning at 10 MHz and AES requires from trigger
obs_duration = 3.225E-6
# Sample at least 1260 points within that window
sampling_interval = obs_duration / 1260
# Configure timebase
(actualSamplingInterval, nSamples, maxSamples) = ps.setSamplingInterval(sampling_interval, obs_duration)
print("Nsamples : ", nSamples)
print("Sampling interval = %f us" % (actualSamplingInterval*nSamples*1E6))

# 50mV range on channel A, AC coupled, 20 MHz BW limit
ps.setChannel('A', 'AC', 0.05, 0.0, enabled=True, BWLimited=True)
# Channel B is trigger
ps.setChannel('B', 'DC', 10.0, 0.0, enabled=True)
ps.setSimpleTrigger('B', 2.0, 'Rising', timeout_ms=2000, enabled=True)

Found the following picoscope:
DriverVersion                 : PS5000A Linux Driver, 2.1.139.6031
USBVersion                    : 3.0
HardwareVersion               : 1
VariantInfo                   : 5244D
BatchAndSerial                : KU687/0175
CalDate                       : 26Apr23
KernelVersion                 : 0.0
DigitalHardwareVersion        : 1
AnalogueHardwareVersion       : 1
PicoFirmwareVersion1          : 1.7.15.0
PicoFirmwareVersion2          : 1.2.34.0
Nsamples :  1612
Sampling interval = 3.224000 us


**CW305 setup**

In [3]:
sbox_dict = {
    "sbox_aes" : "RIJANDAEL",
    "sbox_freyre_1" : "FREYRE_1",
    "sbox_freyre_2" : "FREYRE_2",
    "sbox_freyre_3" : "FREYRE_3",
    "sbox_hussain_6" : "HUSSAIN_6",
    "sbox_ozkaynak_1" : "OZKAYNAK_1",
    "sbox_azam_1" : "AZAM_1",
    "sbox_azam_2" : "AZAM_2",
    "sbox_azam_3" : "AZAM_3"
}

sbox_type = "sbox_azam_3"

In [ ]:
#TARGET_PLATFORM = 'CW305_100t'
#fpga_id = '100t'
# Programming the target with default AES bitstream
#target = cw.target(None, cw.targets.CW305, fpga_id=fpga_id, force = True)

bitstream = r"../rtl/fpga/bitstream/aes/cw305_top_aes_single_round_" + sbox_dict[sbox_type] + r".bit"
target = cw.target(scope, cw.targets.CW305, bsfile=bitstream, force=True)

print(target.is_programmed())
print(bitstream)

target.vccint_set(1.0)
target.pll.pll_enable_set(True)         # enable PLL chip
target.pll.pll_outenable_set(False, 0)  # disable PLL 0
target.pll.pll_outenable_set(True, 1)   # enable PLL 1
target.pll.pll_outenable_set(False, 2)  # disable PLL 2
target.pll.pll_outfreq_set(10E6, 1)     # PLL1 frequency set to 10 MHz
# 1 ms is plenty idling time --> maybe not useful with picoscope (?) 
target.clksleeptime = 1 

True
../rtl/fpga/bitstream/cw305_top_aes_single_round_AZAM_3.bit


**Capture loop**

In [5]:
def pico_capture():
    # Arm the picoscope
    ps.runBlock()
    time.sleep(0.05)
    # Trigger the encryption on Target
    target.fpga_write(target.REG_USER_LED, [0x01])
    target.usb_trigger_toggle()
    ps.waitReady()
    # Capture the trace 
    data = ps.getDataV('A', nSamples, returnOverflow=False)
    return data

In [6]:
def capture_trace(target, sbox_type, keyfix, textfix):
    
    key = bytearray.fromhex(keyfix)
    text = bytearray.fromhex(textfix)
    
    #Generate the expected ciphertext
    text_py = [textfix[i:i+2] for i in range(0, len(textfix), 2)]
    text_py = ['0x' + element for element in text_py]
    text_py = [int(s, 16) for s in text_py]

    ciphertext = AESpy.encrypt(keyfix, text_py, sbox_type)

    # Write key to target
    target.fpga_write(target.REG_CRYPT_KEY, key[::-1])

    # Write plaintext to target
    inputtext = text[::-1]
    target.fpga_write(target.REG_CRYPT_TEXTIN, inputtext)
    
    # Capture the trace 
    data = pico_capture()
    
    # Organize and store data
    response = target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)
    response = response[::-1]
    
    trace_i = Trace(np.array(data), text, response, key)
    
    # Sanity check with expected ciphertext
    assert (list(response) == ciphertext), "Incorrect encryption result!\nGot {}\nExp {}\n".format(list(response), ciphertext)
    
    return data, trace_i

In [7]:
from tqdm.notebook import tnrange

from Crypto.Cipher import AES
from chipwhisperer.common.traces import Trace

from AES import AES as AESpy


project_file_fixed = "../build/sca_TVLA_fixed_" + sbox_dict[sbox_type] + "/sca_test_CW305_TVLA_fixed.cwp"
project_fixed = cw.create_project(project_file_fixed, overwrite=True)
project_file_random = "../build/sca_TVLA_random_" + sbox_dict[sbox_type] + "/sca_test_CW305_TVLA_random.cwp"
project_random = cw.create_project(project_file_random, overwrite=True)

N = 10000        # Number of traces
traces_fixed = []
textin_fixed = []
keys_fixed = []
data_mV_fixed = []

traces_random = []
textin_random = []
keys_random = []
data_mV_random = []

keyfix = "0123456789abcdef123456789abcdef0"
textfix = "da39a3ee5e6b4b0d3255bfef95601890"

keygen = "123456789abcdef123456789abcde0f0"
textrandom = "00000000000000000000000000000000"
keydev = "0123456789abcdef123456789abcdef0"

# Dummy capture call due to bug of using AC coupling
pico_capture()

for i in tnrange(N, desc='Capturing traces'):
    
    # Capture fixed dataset
    data, trace_i = capture_trace(target, sbox_type, keyfix, textfix)
    
    traces_fixed.append(np.array(data))
    data_mV_fixed.append(np.array(data)*1E3)
    
    project_fixed.traces.append(trace_i)
    
    textin_fixed.append(bytearray.fromhex(textfix))
    keys_fixed.append(bytearray.fromhex(keyfix))

    # Capture random dataset
    data, trace_i = capture_trace(target, sbox_type, keydev, textrandom)
    
    traces_random.append(np.array(data))
    data_mV_random.append(np.array(data)*1E3)
    
    project_random.traces.append(trace_i)
    
    textin_random.append(bytearray.fromhex(textrandom))
    keys_random.append(bytearray.fromhex(keydev))
    
    text_py = [textrandom[i:i+2] for i in range(0, len(textrandom), 2)]
    text_py = ['0x' + element for element in text_py]
    text_py = [int(s, 16) for s in text_py]
    ciphertext = AESpy.encrypt(keygen, text_py, sbox_type)
    textrandom = ''.join(f'{byte:02x}' for byte in ciphertext)

project_fixed.save()
project_fixed.close()
project_random.save()
project_random.close()
target.dis()

Capturing traces:   0%|          | 0/10000 [00:00<?, ?it/s]

**TVLA**

In [11]:
import chipwhisperer as cw

sbox_dict = {
    "sbox_aes" : "RIJANDAEL",
    "sbox_freyre_1" : "FREYRE_1",
    "sbox_freyre_2" : "FREYRE_2",
    "sbox_freyre_3" : "FREYRE_3",
    "sbox_hussain_6" : "HUSSAIN_6",
    "sbox_ozkaynak_1" : "OZKAYNAK_1",
    "sbox_azam_1" : "AZAM_1",
    "sbox_azam_2" : "AZAM_2",
    "sbox_azam_3" : "AZAM_3"
}

sbox_type = "sbox_azam_1"

project_file_fixed = "../build/sca_TVLA_fixed_" + sbox_dict[sbox_type] + "/sca_test_CW305_TVLA_fixed.cwp"
project_fixed = cw.open_project(project_file_fixed)
project_file_random = "../build/sca_TVLA_random_" + sbox_dict[sbox_type] + "/sca_test_CW305_TVLA_random.cwp"
project_random = cw.open_project(project_file_random)

traces_fixed = project_fixed.waves[:]
traces_random = project_random.waves[:]

project_fixed.close()
project_random.close()

In [12]:
def welch_T_test(groupA, groupB):
    
    Xa = []
    Xb = []
    Sa = []
    Sb = []
    t_stat = []

    N_trace = len(groupA)
    N_sample = len(groupA[0])

    # Compute the mean
    for i in range(N_sample):
        sumA = 0
        sumB = 0
        for k in range(N_trace):
            sumA += groupA[k][i]
            sumB += groupB[k][i]
        Xa.append(sumA/N_trace)
        Xb.append(sumB/N_trace)

    # Compute the standard deviation
    for i in range(N_sample):
        sumA = 0
        sumB = 0
        for k in range(N_trace):
            sumA += (groupA[k][i] - Xa[i]) ** 2
            sumB += (groupB[k][i] - Xb[i]) ** 2
        Sa.append((sumA/(N_trace-1)) ** 0.5)
        Sb.append((sumB/(N_trace-1)) ** 0.5)

    #Compute the t-statistic
    for i in range(N_sample):
        t_stat.append((Xa[i] - Xb[i])/(((Sa[i] ** 2)/N_trace+(Sb[i] ** 2)/N_trace) ** 0.5))
        
    return t_stat


In [13]:
def var_test(t_stat):
    threshold=4.5
    if t_stat > threshold or t_stat < -threshold:
        return 1
    else:
        return 0

In [14]:
half = int(len(traces_fixed)/2)
traces_fixed_1 = traces_fixed[:half]
traces_fixed_2 = traces_fixed[half:]
traces_random_1 = traces_random[:half]
traces_random_2 = traces_random[half:]

t_stat_1 = welch_T_test(traces_fixed_1, traces_random_1)
t_stat_2 = welch_T_test(traces_fixed_2, traces_random_2)


In [15]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure(plot_width=800)

threshold_plus = [4.5] *len(t_stat_1)
threshold_minus = [-4.5] *len(t_stat_1)

xrange = range(len(t_stat_1))
p.line(xrange, t_stat_1, line_color="red")
p.line(xrange, t_stat_2, line_color="green")
p.line(xrange, threshold_plus, line_color="orange")
p.line(xrange, threshold_minus, line_color="orange")
show(p)

Loading BokehJS ...